In [1]:
import pandas as pd
import numpy as np
import collections #统计
import nltk #自然语言处理包
from nltk.collocations import * 
from gensim.models import word2vec #词向量

D:\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def getPoems(dynasty):
    path = r'C:/Users/HP/新建文件夹/pa_chong/poem/朝代/'+dynasty+r'total.csv'
    f=open(path,'rb')
    content = pd.read_csv(f, header=None)[1:]
    f.close()
    poem=content[3]
    return poem

In [3]:
def getWords(dynasty):
    path = r'C:/Users/HP/新建文件夹/pa_chong/poem/朝代/'+dynasty+r'.txt'
    # 从文件读取分词
    with open(path,encoding='utf-8') as f:
        line = f.readline()
    words = list(line)
    return words

In [43]:
def load():
    path = r'C:/Users/HP/新建文件夹/pa_chong/poem/total.csv'
    f=open(path,'rb')
    content = pd.read_csv(f, header=None)[1:]
    f.close()
    poem=content[3]
    return poem

In [46]:
def test():
    path = r'C:/Users/HP/新建文件夹/pa_chong/poem/杜甫.csv'
    f=open(path,'rb')

    content = pd.read_csv(f, header=None,encoding='gb18030')[1:]
    f.close()
    poem=content[3]
    return poem

In [49]:
dynasty = '金朝'
# poems = getPoems(dynasty)
# words = getWords(dynasty)
poems = test()

In [21]:
total=load()

In [7]:
model=word2vec.Word2Vec(list(total),min_count=3, size=100, window=5, workers=4)

In [18]:
 model.wv.most_similar('思',topn=20)

[('怀', 0.594771683216095),
 ('忆', 0.48607704043388367),
 ('逢', 0.4842267334461212),
 ('念', 0.48203808069229126),
 ('期', 0.4721718728542328),
 ('情', 0.44191285967826843),
 ('魂', 0.44008615612983704),
 ('愁', 0.4336218535900116),
 ('恨', 0.4227339029312134),
 ('忡', 0.42057764530181885),
 ('慰', 0.41989946365356445),
 ('忘', 0.41428232192993164),
 ('感', 0.4072200059890747),
 ('望', 0.392672061920166),
 ('悰', 0.38949477672576904),
 ('肠', 0.3860398232936859),
 ('怊', 0.38501790165901184),
 ('悽', 0.3849114775657654),
 ('忖', 0.3848033547401428),
 ('怏', 0.38475728034973145)]

In [80]:
def get_emotion(emotion,layer1,layer2,re):
    dit = {}
    try:
        dict_emotion={emotion:1}
        for i in model.wv.most_similar(emotion,topn=layer1): 
            dict_emotion[i[0]]=dict_emotion.get(i[0],0)+i[1]
            for j in model.wv.most_similar(i[0],topn=layer2):
#                 print(i,j)
                dict_emotion[j[0]]=dict_emotion.get(j[0],0)+i[1]*j[1]
        sortlist=sorted(dict_emotion.items(),key = lambda x:x[1],reverse = True)[:re]
        
        for i in sortlist:
            dit[i[0]] = i[1]
        return dit,sortlist
    except Exception as e:
#         print(e)
        return dit,0

In [81]:
def getScore(poem,layer1,layer2,re):
#     emotions=['喜','怒','忧','思','悲','恐','惊']
    emotions=['悲','惧','乐','怒','思','喜','忧']
    type_probability={}
    for emotion in emotions:
        dit,temp=get_emotion(emotion,layer1,layer2,re)
        score=0
        for i in poem:
#             print(i)
            score+=dit.get(i,0)
        score=score/len(poem)
        type_probability[emotion]=score
    type_probability.values()
#     print(type_probability)
    total_score=sum(list(type_probability.values()))
    if total_score==0:
        return '未知分类',1
    most =sorted(type_probability.items(),key = lambda x:x[1],reverse = True)[0]

    return most[0],most[1]/total_score

In [82]:
def distribution(poems,layer1=20,layer2=5,re=100):
    distribute_dict={}
    i=0
    length = len(poems)
    for poem in poems:
        i=i+1
        t,s=getScore(poem,layer1,layer2,re)
        distribute_dict[t]=distribute_dict.get(t,0)+1
        if i%100==0:
            print(str(i)+"       -------------"+str(i/length))
            
    return distribute_dict

In [75]:
poems

1      八月秋高风怒号，卷我屋上三重茅。茅飞渡江洒江郊，高者挂罥长林梢，下者飘转沉塘坳。南村群童欺我...
2       好雨知时节，当春乃发生。随风潜入夜，润物细无声。野径云俱黑，江船火独明。晓看红湿处，花重锦官城。
3      岱宗夫如何？齐鲁青未了。造化钟神秀，阴阳割昏晓。荡胸生曾云，决眦入归鸟。( 曾 通：层)会当...
4                       两个黄鹂鸣翠柳，一行白鹭上青天。窗含西岭千秋雪，门泊东吴万里船。
5      剑外忽传收蓟北，初闻涕泪满衣裳。却看妻子愁何在，漫卷诗书喜欲狂。白日放歌须纵酒，青春作伴好还...
6      暮投石壕村，有吏夜捉人。老翁逾墙走，老妇出门看。吏呼一何怒，妇啼一何苦。听妇前致词，三男邺城...
7      风急天高猿啸哀，渚清沙白鸟飞回。无边落木萧萧下，不尽长江滚滚来。万里悲秋常作客，百年多病独登...
8                       黄四娘家花满蹊，千朵万朵压枝低。留连戏蝶时时舞，自在娇莺恰恰啼。
9      戍鼓断人行，边秋一雁声。(边秋 一作:秋边)露从今夜白，月是故乡明。有弟皆分散，无家问死生。...
10     丞相祠堂何处寻，锦官城外柏森森。映阶碧草自春色，隔叶黄鹂空好音。三顾频烦天下计，两朝开济老臣...
11                     岐王宅里寻常见，崔九堂前几度闻。 正是江南好风景，落花时节又逢君。
12      昔闻洞庭水，今上岳阳楼。吴楚东南坼，乾坤日夜浮。亲朋无一字，老病有孤舟。戎马关山北，凭轩涕泗流。
13                      黄师塔前江水东，春光懒困倚微风。桃花一簇开无主，可爱深红爱浅红？
14     迟日江山丽，春风花草香。泥融飞燕子，沙暖睡鸳鸯。 江碧鸟逾白，山青花欲燃。今春看又过，何日是归年。
15                    功盖三分国，名成八阵图。（名成 一作：名高）江流石不转，遗恨失吞吴。
16                     锦城丝管日纷纷，半入江风半入云。 此曲只应天上有，人间能得几回闻。
17      细草微风岸，危樯独夜舟。星垂平野阔，月涌大江流。名岂文章著，官应老病休。飘飘何所似，天地一沙鸥。
18                             

In [83]:
fenbu = distribution(poems)

100       -------------0.1001001001001001
200       -------------0.2002002002002002
300       -------------0.3003003003003003
400       -------------0.4004004004004004
500       -------------0.5005005005005005
600       -------------0.6006006006006006
700       -------------0.7007007007007007
800       -------------0.8008008008008008
900       -------------0.9009009009009009


In [52]:
fenbu

{'乐': 252, '喜': 104, '忧': 13, '怒': 44, '思': 375, '悲': 167, '惧': 36, '未知分类': 8}

In [60]:
fenbu 50 5 100

{'乐': 268, '喜': 149, '忧': 14, '怒': 39, '思': 334, '悲': 156, '惧': 32, '未知分类': 7}

In [64]:
fenbu 100 1 100

{'乐': 270, '喜': 197, '忧': 70, '怒': 33, '思': 210, '悲': 154, '惧': 36, '未知分类': 29}

In [84]:
fenbu 

{'乐': 152, '喜': 161, '忧': 44, '怒': 42, '思': 368, '悲': 171, '惧': 20, '未知分类': 41}